In [1]:
!nvidia-smi # 查看GPU信息

Invalid combination of input arguments. Please run 'nvidia-smi -h' for help.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#进入需要训练项目的文件夹
%cd /content/drive/My Drive/Colab Notebooks/ML2020_spring_crack_detection/CrackU-Net

In [ ]:
!pip install torch torchvision

In [1]:
from models_crack_unet import SegmentNet, weights_init_normal
from dataset_crack_unet import CFDDataset

import torch.nn as nn
import torch

from torchvision import datasets
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

import os
import sys
import argparse
import time
import PIL.Image as Image

import numpy as np

In [2]:
# 在ipynb文件中，parse的创建用函数来创建
# 直接用parser=parser = argparse.ArgumentParser() 来创建之后调用会报错

def get_arguments():
    parser = argparse.ArgumentParser()

    parser.add_argument("--cuda", type=bool, default=True, help="number of gpu")
    parser.add_argument("--gpu_num", type=int, default=1, help="number of gpu")
    parser.add_argument("--worker_num", type=int, default=0, help="number of input workers") # 只有一个GPU,default=0表示单进程加载
    parser.add_argument("--batch_size", type=int, default=2, help="batch size of input")
    parser.add_argument("--lr", type=float, default=0.0005, help="adam: learning rate")
    parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
    parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")

    parser.add_argument("--begin_epoch", type=int, default=0, help="begin_epoch")
    parser.add_argument("--end_epoch", type=int, default=101, help="end_epoch")

    parser.add_argument("--need_test", type=bool, default=True, help="need to test")
    parser.add_argument("--test_interval", type=int, default=10, help="interval of test")
    parser.add_argument("--need_save", type=bool, default=True, help="need to save")
    parser.add_argument("--save_interval", type=int, default=30, help="interval of save weights")


    parser.add_argument("--img_width", type=int, default=480, help="size of image width")
    parser.add_argument("--img_height", type=int, default=320, help="size of image height")
    
    return parser.parse_args(args=[])

opt = get_arguments()

print(opt)

Namespace(b1=0.5, b2=0.999, batch_size=1, begin_epoch=0, cuda=True, end_epoch=101, gpu_num=1, img_height=320, img_width=480, lr=0.0005, need_save=True, need_test=True, save_interval=30, test_interval=10, worker_num=0)


In [3]:
dataSetRoot = "../Data" 

In [5]:
# Build nets
segment_net = SegmentNet(init_weights=True)

# Loss functions
criterion_segment  = torch.nn.MSELoss()
# criterion_segment = torch.nn.BCEWithLogitsLoss()
# criterion_segment  = torch.nn.BCELoss()

In [6]:
# 选择训练环境和参数
if opt.cuda:
    segment_net = segment_net.cuda()
    criterion_segment.cuda()

if opt.gpu_num > 1:
    segment_net = torch.nn.DataParallel(segment_net, device_ids=list(range(opt.gpu_num)))

if opt.begin_epoch != 0:
    # Load pretrained models
    segment_net.load_state_dict(torch.load("./saved_models/segment_net_%d.pth" % (opt.begin_epoch)))
else:
    # Initialize weights
    segment_net.apply(weights_init_normal)

In [7]:
# Optimizers
optimizer_seg = torch.optim.Adam(segment_net.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [8]:
transforms_ = transforms.Compose([
    transforms.Resize((opt.img_height, opt.img_width), Image.BICUBIC),
    transforms.ToTensor()
])

transforms_mask = transforms.Compose([
    transforms.Resize((opt.img_height, opt.img_width)), 
    transforms.ToTensor()
])

In [9]:
trainCFDloader = DataLoader(
    CFDDataset(dataSetRoot, transforms_=transforms_, transforms_mask= transforms_mask, subFold="CFD", isTrain=True),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.worker_num,
)

In [10]:
for epoch in range(opt.begin_epoch, opt.end_epoch):

    iterCFD = trainCFDloader.__iter__()

    lenNum = len(trainCFDloader)

    segment_net.train()
    # train *****************************************************************

    # 存储每一个epoch的总损失和总精度
    train_loss_sum, train_acc_sum, batch_count = 0.0, 0.0, 0.0

    for i in range(0, lenNum):
        #if i % 2 == 0:
            #batchData = iterOK.__next__()
            #idx, batchData = enumerate(trainOKloader)
        #else :
        #    batchData = iterNG.__next__()
            #idx, batchData = enumerate(trainNGloader)
        
        batchData = iterCFD.__next__()

        if opt.cuda:
            img = batchData["img"].cuda()
            mask = batchData["mask"].cuda()
        else:
            img = batchData["img"]
            mask = batchData["mask"]
    
        optimizer_seg.zero_grad()

        rst = segment_net(img)
        seg = rst["seg"]
        # print(seg.size())  输出：torch.Size([2, 1, 88, 32])

        loss_seg = criterion_segment(seg, mask)
        loss_seg.backward()
        optimizer_seg.step()

        train_loss_sum += loss_seg.item() 
    
        
         # 计算seg精度
        net_seg = seg.clone().flatten()    # 训练值
        mask_seg = mask.clone().flatten()  #真实值
        
        right_seg = torch.eq(net_seg, mask_seg).sum().float().item()
        total_num = float(mask.clone().flatten().size()[0])
        
        batch_acc = right_seg/total_num
        train_acc_sum += batch_acc
        
        batch_count += 1

    print("[Epoch {0}/{1}], [loss:{2}], [accuracy:{3}]".format(epoch, opt.end_epoch, train_loss_sum/batch_count, train_acc_sum/batch_count))


    # save parameters *****************************************************************
    if opt.need_save and epoch % opt.save_interval == 0 and epoch >= opt.save_interval:

        save_path_str = "./saved_models"
        if os.path.exists(save_path_str) == False:
            os.makedirs(save_path_str, exist_ok=True)

        torch.save(segment_net.state_dict(), "%s/segment_net_%d.pth" % (save_path_str, epoch))
        print("save weights ! epoch = %d" %epoch)
        pass

[Epoch 0/101], [loss:0.022734979012360174], [accuracy:0.03990668402777777]


KeyboardInterrupt: 